In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
%run PARAMETERS.py
NAME = 'APG'
MCMC_STEPS = 5
PATH = NAME + '-bmnist-%dsamples' % (SAMPLE_SIZE)
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')

In [3]:
from model_operations import Init_models, Save_models
models, optimizer = Init_models(K=K,
                                D=D,
                                FRAME_PIXELS=FRAME_PIXELS, 
                                DIGIT_PIXELS=DIGIT_PIXELS, 
                                HIDDEN_LIST=HIDDEN_LIST, 
                                CUDA=CUDA, 
                                device=DEVICE, 
                                lr=LEARNING_RATE, 
                                RESTORE=False, 
                                PATH=None)

from crop import *
crop = Crop(digit_size=28, 
            frame_size=64, 
            CUDA=CUDA, 
            device=DEVICE)

In [4]:
from apg import APG
train(models=models, 
      objective=APG, 
      optimizer=optimizer, 
      data_path=data_path, 
      mcmc_steps=MCMC_STEPS,
      mnist_mean=mnist_mean,
      crop=crop,
      Train_Params=Train_Params,
      CUDA=CUDA,
      device=DEVICE,
      path=PATH)

Completed  in (15s),  phi_loss: 21.554,  theta_loss: 22732.670,  ess: 1.041
Completed  in (15s),  phi_loss: -25.844,  theta_loss: 5269.885,  ess: 1.077
Completed  in (15s),  phi_loss: -36.003,  theta_loss: 4307.503,  ess: 1.074
Completed  in (16s),  phi_loss: -44.046,  theta_loss: 4038.683,  ess: 1.069
Completed  in (15s),  phi_loss: -47.617,  theta_loss: 3866.039,  ess: 1.072
Completed  in (15s),  phi_loss: -51.706,  theta_loss: 3769.238,  ess: 1.063
Completed  in (15s),  phi_loss: -52.636,  theta_loss: 3728.744,  ess: 1.069
Completed  in (15s),  phi_loss: -56.585,  theta_loss: 3662.133,  ess: 1.081
Completed  in (15s),  phi_loss: -58.071,  theta_loss: 3769.497,  ess: 1.068
Completed  in (15s),  phi_loss: -59.493,  theta_loss: 3630.530,  ess: 1.087
Completed  in (15s),  phi_loss: -62.033,  theta_loss: 3539.443,  ess: 1.086
Completed  in (15s),  phi_loss: -59.100,  theta_loss: 3568.382,  ess: 1.074
Completed  in (15s),  phi_loss: -60.498,  theta_loss: 3496.417,  ess: 1.076
Completed  i

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-e6d037a0c77a>", line 12, in <module>
    path=PATH)
  File "../training.py", line 36, in train
    metrics = objective(models, K, D, frames, mcmc_steps, mnist_mean, crop, tj_b, tj_std)
  File "../objectives/apg.py", line 20, in APG
    phi_loss_where, theta_loss_where, ess_where, w_where, z_where = Update_where(tj_b, tj_std, enc_coor, dec_digit, frames, crop, z_what, z_where_old=z_where)
  File "../forward_backward.py", line 47, in Update_where
    q_b_where = enc_coor(frames=frames, digit=digit, sampled=False, z_where_old=z_where_old)
  File "/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "../models/enc_coor.py", line 49, in forward
    q.normal(loc=q_mean, sc

KeyboardInterrupt: 

In [ ]:
Save_models(models, path=PATH)